# InterLab example: Gradually summarizating memory for an LLM actor

Note that you need to have the LLM API keys [stored in an `.env` file](https://github.com/theskumar/python-dotenv#getting-started) (recommended), or as environmetn variables.

In [5]:
import dotenv
import json
import langchain, langchain_openai
from pydantic.dataclasses import dataclass, Field
import pandas as pd
from matplotlib import pyplot as plt

import interlab
from interlab import actor
import treetrace
from treetrace import FileStorage, TracingNode

dotenv.load_dotenv()

True

## Scenario texts

Complete initial prompts.

In [6]:
STORY = """
Once upon a time in the middle of winter, when the flakes of snow were falling like feathers from the clouds, a Queen sat at her palace window, which had an ebony black frame, stitching her husband’s shirts. While she was thus engaged and looking out at the snow she pricked her finger, and three drops of blood fell upon the snow. Now the red looked so well upon the white that she thought to herself, “Oh, that I had a child as white as this snow, as red as this blood, and as black as the wood of this frame!” Soon afterwards a little daughter came to her, who was as white as snow, and with cheeks as red as blood, and with hair as black as ebony, and from this she was named “Snow-White.” And at the same time her mother died.
About a year afterwards the King married another wife, who was very beautiful, but so proud and haughty that she could not bear anyone to be better-looking than herself. She owned a wonderful mirror, and when she stepped before it and said: “Mirror, mirror on the wall, Who is the fairest of us all?” it replied: “The Queen is the fairest of the day.” Then she was pleased, for she knew that the mirror spoke truly.
Little Snow-White, however, grew up, and became prettier and prettier, and when she was seven years old she was as fair as the noonday, and more beautiful than the Queen herself. When the Queen now asked her mirror: “Mirror, mirror on the wall, Who is the fairest of us all?” it replied: “The Queen was fairest yesterday; Snow-White is the fairest, now, they say.”
This answer so angered the Queen that she became quite yellow with envy. From that hour, whenever she saw Snow-White, her heart was hardened against her, and she hated the little girl. Her envy and jealousy increased so that she had no rest day or night, and she said to a Huntsman, “Take the child away into the forest. I will never look upon her again. You must kill her, and bring me her heart and tongue for a token.”
The Huntsman listened and took the maiden away, but when he drew out his knife to kill her, she began to cry, saying, “Ah, dear Huntsman, give me my life! I will run into the wild forest, and never come home again.”
This speech softened the Hunter’s heart, and her beauty so touched him that he had pity on her and said, “Well, run away then, poor child.” But he thought to himself, “The wild beasts will soon devour you.” Still he felt as if a stone had been lifted from his heart, because her death was not by his hand. Just at that moment a young boar came roaring along to the spot, and as soon as he clapped eyes upon it the Huntsman caught it, and, killing it, took its tongue and heart and carried them to the Queen, for a token of his deed.
But now poor little Snow-White was left motherless and alone, and overcome with grief, she was bewildered at the sight of so many trees, and knew not which way to turn. She ran till her feet refused to go farther, and as it was getting dark, and she saw a little house near, she entered in to rest. In this cottage everything was very small, but very neat and elegant. In the middle stood a little table with a white cloth over it, and seven little plates upon it, each plate having a spoon and a knife and a fork, and there were also seven little mugs. Against the wall were seven little beds arranged in a row, each covered with snow-white sheets.
Little Snow-White, being both hungry and thirsty, ate a little morsel of porridge out of each plate, and drank a drop or two of wine out of each mug, for she did not wish to take away the whole share of anyone. After that, because she was so tired, she laid herself down on one bed, but it did not suit; she tried another, but that was too long; a fourth was too short, a fifth too hard. But the seventh was just the thing; and tucking herself up in it, she went to sleep, first saying her prayers as usual.
When it became quite dark the owners of the cottage came home, seven Dwarfs, who dug for gold and silver in the mountains. They first lighted seven little lamps, and saw at once—for they lit up the whole room—that somebody had been in, for everything was not in the order in which they had left it.
The first asked, “Who has been sitting on my chair?” The second, “Who has been eating off my plate?” The third said, “Who has been nibbling at my bread?” The fourth, “Who has been at my porridge?” The fifth, “Who has been meddling with my fork?” The sixth grumbled out, “Who has been cutting with my knife?” The seventh said, “Who has been drinking out of my mug?”
Then the first, looking round, began again, “Who has been lying on my bed?” he asked, for he saw that the sheets were tumbled. At these words the others came, and looking at their beds cried out too, “Some one has been lying in our beds!” But the seventh little man, running up to his, saw Snow-White sleeping in it; so he called his companions, who shouted with wonder and held up their seven lamps, so that the light fell upon the little girl.
“Oh, heavens! oh, heavens!” said they; “what a beauty she is!” and they were so much delighted that they would not awaken her, but left her to sleep, and the seventh Dwarf, in whose bed she was, slept with each of his fellows one hour, and so passed the night.
As soon as morning dawned Snow-White awoke, and was quite frightened when she saw the seven little men; but they were very friendly, and asked her what she was called. “My name is Snow-White,” was her reply. 
“Why have you come into our cottage?” they asked. Then she told them how her stepmother would have had her killed, but the Huntsman had spared her life, and how she had wandered about the Whole day until at last she had found their house.
When her tale was finished the Dwarfs said, “Will you look after our household—be our cook, make the beds, wash, sew, and knit for us, and keep everything in neat order? If so, we will keep you here, and you shall want for nothing.” And Snow-White answered, “Yes, with all my heart and will.” And so she remained with them, and kept their house in order.
In the morning the Dwarfs went into the mountains and searched for silver and gold, and in the evening they came home and found their meals ready for them. During the day the maiden was left alone, and therefore the good Dwarfs warned her and said, “Be careful of your stepmother, who will soon know of your being here. So let nobody enter the cottage.”
The Queen meanwhile, supposing that she had eaten the heart and tongue of her stepdaughter, believed that she was now above all the most beautiful woman in the world. One day she stepped before her mirror, and said: “Mirror, mirror on the wall, Who is the fairest of us all?” and it replied: “The Queen was fairest yesterday; Snow-White is fairest now, they say. The Dwarfs protect her from thy sway Amid the forest, far away.”
This reply surprised her, but she knew that the mirror spoke the truth. She knew, therefore, that the Huntsman had deceived her, and that Snow-White was still alive. So she dyed her face and clothed herself as a pedler woman, so that no one could recognize her, and in this disguise she went over the seven hills to the house of the seven Dwarfs. She knocked at the door of the hut, and called out, “Fine goods for sale! beautiful goods for sale!”
Snow-White peeped out of the window and said, “Good day, my good woman; what have you to sell?” “Fine goods, beautiful goods!” she replied. “Stays of all colors.” And she held up a pair which were made of many-colored silks.
“I may let in this honest woman,” thought Snow-White; and she unbolted the door and bargained for one pair of stays. “You can’t think, my dear, how they become you!” exclaimed the old woman. “Come, let me lace them up for you.”
Snow-White suspected nothing, and let her do as she wished, but the old woman laced her up so quickly and so tightly that all her breath went, and she fell down like one dead. “Now,” thought the old woman to herself, hastening away, “now am I once more the most beautiful of all!”
At eventide, not long after she had left, the seven Dwarfs came home, and were much frightened at seeing their dear little maid lying on the ground, and neither moving nor breathing, as if she were dead. They raised her up, and when they saw that she was laced too tight they cut the stays to pieces, and presently she began to breathe again, and  little by little she revived. When the Dwarfs now heard what had taken place, they said, “The old pedler woman was no other than your wicked stepmother. Take more care of yourself, and let no one enter when we are not with you.”
Meanwhile, the Queen had reached home, and, going before her mirror, she repeated her usual words: “Mirror, mirror on the wall, Who is the fairest of us all?” and it replied as before: “The Queen was fairest yesterday; Snow-White is fairest now, they say. The Dwarfs protect her from thy sway Amid the forest, far away.”
As soon as it had finished, all her blood rushed to her heart, for she was so angry to hear that Snow-White was yet living. “But now,” thought she to herself, “will I make something which shall destroy her completely.” Thus saying, she made a poisoned comb by arts which she understood, and then, disguising herself, she took the form of an old widow. She went over the seven hills to the house of the seven Dwarfs, and knocking at the door, called out, “Good wares to sell to-day!”
Snow-White peeped out and said, “You must go farther, for I dare not let you in.” “But still you may look,” said the old woman, drawing out her poisoned comb and holding it up. The sight of this pleased the maiden so much that she allowed herself to be persuaded, and opened the door. As soon as she had bought something the old woman said, “Now let me for once comb your hair properly,” and Snow-White consented. But scarcely was the comb drawn through the hair when the poison began to work, and the maiden fell down senseless.
“You pattern of beauty,” cried the wicked Queen, “it is now all over with you.” And so saying, she departed. Fortunately, evening soon came, and the seven Dwarfs returned, and as soon as they saw Snow-White lying, like dead, upon the ground, they suspected the Queen, and discovering the poisoned comb, they immediately drew it out. Then the maiden very soon revived and told them all that had happened. So again they warned her against the wicked stepmother, and bade her open the door to nobody.
Meanwhile the Queen, on her arrival home, had again consulted her mirror, and received the same answer as twice before. This made her tremble and foam with rage and jealousy, and she swore that Snow-White should die if it cost her her own life. Thereupon she went into an inner secret chamber where no one could enter, and made an apple of the most deep and subtle poison. Outwardly it looked nice enough, and had rosy cheeks which would make the mouth of everyone who looked at it water; but whoever ate the smallest piece of it would surely die. As soon as the apple was ready the Queen again dyed her face, and clothed herself like a peasant’s wife, and then over the seven mountains to the house of the seven Dwarfs she made her way.
She knocked at the door, and Snow-White stretched out her head and said, “I dare not let anyone enter; the seven Dwarfs have forbidden me.” “That is hard on me,” said the old woman, “for I must take back my apples; but there is one which I will give you.” “No,” answered Snow-White; “no, I dare not take it.”
“What! are you afraid of it?” cried the old woman. “There, see—I will cut the apple in halves; do you eat the red cheeks, and I will eat the core.” (The apple was so artfully made that the red cheeks alone were poisoned.) Snow-White very much wished for the beautiful apple, and when she saw the woman eating the core she could no longer resist, but, stretching out her hand, took the poisoned part. Scarcely had she placed a piece in her mouth when she fell down dead upon the ground. Then the Queen, looking at her with glittering eyes, and laughing bitterly, exclaimed, “White as snow, red as blood, black as ebony! This time the Dwarfs cannot reawaken you.”
When she reached home and consulted her mirror—“Mirror, mirror on the wall, Who is the fairest of us all?”  it answered:  “The Queen is fairest of the day.” Then her envious heart was at rest, as peacefully as an envious heart can rest.
When the little Dwarfs returned home in the evening they found Snow-White lying on the ground, and there appeared to be no life in her body; she seemed to be quite dead. They raised her up, and tried if they could find anything poisonous. They unlaced her, and even uncombed her hair, and washed her with water and with wine. But nothing availed: the dear child was really and truly dead.
Then they laid her upon a bier, and all seven placed themselves around it, and wept and wept for three days without ceasing. Then they prepared to bury her. But she looked still fresh and life-like, and even her red cheeks had not deserted her, so they said to one another, “We cannot bury her in the black ground.” Then they ordered a case to be made of glass. In this they could see the body on all sides, and the Dwarfs wrote her name with golden letters upon the glass, saying that she was a King’s daughter. Now they placed the glass case upon the ledge on a rock, and one of them always remained by it watching. Even the birds bewailed the loss of Snow-White; first came an owl, then a raven, and last of all a dove.
For a long time Snow-White lay peacefully in her case, and changed not, but looked as if she were only asleep, for she was still white as snow, red as blood, and black-haired as ebony. By and by it happened that a King’s son was traveling in the forest, and came to the Dwarfs’ house to pass the night. He soon saw the glass case upon the rock, and the beautiful maiden lying within, and he read also the golden inscription.
When he had examined it, he said to the Dwarfs, “Let me have this case, and I will pay what you like for it.” But the Dwarfs replied, “We will not sell it for all the gold in the world.”
“Then give it to me,” said the Prince; “for I cannot live without Snow-White. I will honor and protect her as long as I live.” When the Dwarfs saw that he was so much in earnest, they pitied him, and at last gave him the case, and the Prince ordered it to be carried away on the shoulders of his attendants. Presently it happened that they stumbled over a rut, and with the shock the piece of poisoned apple which lay in Snow-White’s mouth fell out. Very soon she opened her eyes, and raising the lid of the glass case, she rose up and asked, “Where am I?”
Full of joy, the Prince answered, “You are safe with me.” And he told to her what she had suffered, and how he would rather have her than any other for his wife, and he asked her to accompany him home to the castle of the King his father. Snow-White consented, and when they arrived there they were married with great splendor and magnificence.
Snow-White’s stepmother was also invited to the wedding, and when she was dressed in all her finery to go, she first stepped in front of her mirror and asked: “Mirror, mirror on the wall, Who is the fairest of us all?” and it replied: “The Queen was fairest yesterday; The Prince’s bride is now, they say.”
At these words the Queen was in a fury, and was so terribly mortified that she knew not what to do with herself. At first she resolved not to go to the wedding, but she could not resist the wish to see the Princess. So she went; but as soon as she saw the bride she recognized Snow-White, and was so terrified with rage and astonishment that she rushed out of the castle and was never heard of again.
"""

MESSAGES = STORY.split("\n")

## Using summarizing memory

Here we add one line at a time, always showing the current state of the memory ad level of summarization
of each memory item: L0 is just a single original item, Ln means $2^n$ items summarized into one.

Below this long output, you can also find the treetrace browser for the run.

In [13]:
import interlab.actor.memory.experimental.summarizing_memory

model = langchain_openai.ChatOpenAI()
mem = interlab.actor.memory.experimental.summarizing_memory.SummarizingMemory(
    model=model, token_limit=500, one_message_limit=150, summary_limit=100)

with TracingNode(f"summarizing_memory") as c:
    for m in MESSAGES:
        mem.add_memory(m)
        print("\n\n#############################\n\n" + "\n\n".join(f"[L{i.level}, len={i.token_count}] {i.memory}" for i in mem.items))



#############################

[L0, len=1] 


[2024-01-20 20:31:50,615] INFO(httpx): HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"




#############################

[L0, len=1] 

[L0, len=76] In the middle of winter, a Queen pricks her finger while stitching her husband's shirts and three drops of blood fall onto the snow. She wishes for a child as white as snow, as red as blood, and as black as ebony. Shortly after, she gives birth to a daughter named Snow-White who possesses these qualities. Unfortunately, the Queen dies during childbirth.


#############################

[L0, len=1] 

[L0, len=76] In the middle of winter, a Queen pricks her finger while stitching her husband's shirts and three drops of blood fall onto the snow. She wishes for a child as white as snow, as red as blood, and as black as ebony. Shortly after, she gives birth to a daughter named Snow-White who possesses these qualities. Unfortunately, the Queen dies during childbirth.

[L0, len=95] About a year afterwards the King married another wife, who was very beautiful, but so proud and haughty that she could not bear anyone to be better-lookin

[2024-01-20 20:31:51,542] INFO(httpx): HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[2024-01-20 20:31:52,509] INFO(httpx): HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"




#############################

[L1, len=41] In winter, a Queen pricks her finger and wishes for a child with snow-white, blood-red, and ebony-black qualities. She gives birth to Snow-White, but tragically dies during childbirth.

[L1, len=63] After the King married a proud and haughty second wife, she owned a magical mirror that always told her she was the fairest. But as Snow-White grew up, she became even more beautiful than the Queen. When the Queen asked the mirror, it revealed that Snow-White was now the fairest.

[L0, len=96] This answer so angered the Queen that she became quite yellow with envy. From that hour, whenever she saw Snow-White, her heart was hardened against her, and she hated the little girl. Her envy and jealousy increased so that she had no rest day or night, and she said to a Huntsman, “Take the child away into the forest. I will never look upon her again. You must kill her, and bring me her heart and tongue for a token.”

[L0, len=55] The Huntsman listened an

...



#############################

[L4, len=64] After the death of the Queen, Snow-White is born and grows up to be extremely beautiful. Her jealous stepmother orders a Huntsman to kill her, but he lets her go. Snow-White finds a cottage where she meets seven dwarfs who are enchanted by her beauty and allow her to stay with them.

[L3, len=61] The Dwarfs warn Snow-White about her stepmother. The stepmother disguises herself and tricks Snow-White into wearing tight stays, making her fall unconscious. The Dwarfs find her and cut the stays. The Queen discovers Snow-White is alive and guarded by the Dwarfs.

[L3, len=91] In an attempt to seek revenge on Snow-White, the evil queen disguises herself as an elderly widow and gives Snow-White a poisoned comb, causing her to collapse. Later, the queen offers Snow-White a poisoned apple, which she initially refuses but eventually eats, resulting in her death. The queen is pleased with her revenge, believing the dwarfs cannot save Snow-White. The dw

[2024-01-20 20:32:42,422] INFO(httpx): HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[2024-01-20 20:32:43,447] INFO(httpx): HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"




#############################

[L4, len=64] After the death of the Queen, Snow-White is born and grows up to be extremely beautiful. Her jealous stepmother orders a Huntsman to kill her, but he lets her go. Snow-White finds a cottage where she meets seven dwarfs who are enchanted by her beauty and allow her to stay with them.

[L3, len=61] The Dwarfs warn Snow-White about her stepmother. The stepmother disguises herself and tricks Snow-White into wearing tight stays, making her fall unconscious. The Dwarfs find her and cut the stays. The Queen discovers Snow-White is alive and guarded by the Dwarfs.

[L3, len=91] In an attempt to seek revenge on Snow-White, the evil queen disguises herself as an elderly widow and gives Snow-White a poisoned comb, causing her to collapse. Later, the queen offers Snow-White a poisoned apple, which she initially refuses but eventually eats, resulting in her death. The queen is pleased with her revenge, believing the dwarfs cannot save Snow-White. The dw

[2024-01-20 20:32:44,510] INFO(httpx): HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"




#############################

[L4, len=64] After the death of the Queen, Snow-White is born and grows up to be extremely beautiful. Her jealous stepmother orders a Huntsman to kill her, but he lets her go. Snow-White finds a cottage where she meets seven dwarfs who are enchanted by her beauty and allow her to stay with them.

[L3, len=61] The Dwarfs warn Snow-White about her stepmother. The stepmother disguises herself and tricks Snow-White into wearing tight stays, making her fall unconscious. The Dwarfs find her and cut the stays. The Queen discovers Snow-White is alive and guarded by the Dwarfs.

[L3, len=91] In an attempt to seek revenge on Snow-White, the evil queen disguises herself as an elderly widow and gives Snow-White a poisoned comb, causing her to collapse. Later, the queen offers Snow-White a poisoned apple, which she initially refuses but eventually eats, resulting in her death. The queen is pleased with her revenge, believing the dwarfs cannot save Snow-White. The dw

# TreeTrace browser

The trace is only stored in this notebook along with the trace browser. Note the widget is not displayed in GitHub notebook preview.

In [14]:
c.display()